# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [100]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [101]:
# your code here
orders = pd.read_csv('Orders.zip', compression='zip', header=0, sep=',', quotechar='"')

orders.head()

,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34


---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [102]:
# your code here
#group orders by clients and sum all amount spent by each one
clients = orders.groupby(['CustomerID','Country']).agg({'amount_spent':'sum'})
clients

,,amount_spent
CustomerID,Country,
12346,United Kingdom,77183.60
12347,Iceland,4310.00
12348,Finland,1797.24
12349,Italy,1757.55
12350,Norway,334.40
...,...,...
18280,United Kingdom,180.60
18281,United Kingdom,80.82
18282,United Kingdom,178.05


In [103]:
#Find and convert the 75% percentile and 95% percentile
cond1 = float(clients.quantile(0.75))
cond2 = float(clients.quantile(0.95))

In [113]:
#specific function with the parameters to determinated the type of client based on percentile values
def client(x):
    if x>=cond2:
        return 'VIP'
    elif (x<cond2) & (x>=cond1):
        return 'Preferred'
    else:
        return 'Normal'

clients['Type of Client'] = clients['amount_spent'].apply(lambda x: client(x))
clients['Type of Client'].unique()
clients.groupby('Type of Client').agg({'Type of Client':'count'})

,Type of Client
Type of Client,
Normal,3260
Preferred,869
VIP,218


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [128]:
# your code here
#clients.filter(like='Vip',axis=1) #???
#clients.groupby(['Country','Type of Client']).agg({'Type of Client':'count'}) #shows all kind of clients for each Country

clients['Vip'] = clients['Type of Client'].apply(lambda x: True if x=='VIP' else False)
#clients['Vip + Preferred'].unique() #checking if we have True and False values

clients[clients['Vip']==True].groupby(['Country']).agg({'Vip':'count'}).sort_values(by='Vip',ascending=False).head(1)

,Vip
Country,
United Kingdom,178


## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [132]:
# your code here
clients['Vip + Preferred'] = clients['Type of Client'].apply(lambda x: True if (x=='VIP') | (x=='Preferred') else False)
#clients['Vip + Preferred'].unique() #checking if we have True and False values
clients
clients[clients['Vip + Preferred']==True].groupby('Country').agg({'Vip + Preferred':'count'}).sort_values(by='Vip + Preferred',ascending=False).head(1)

,Vip + Preferred
Country,
United Kingdom,934
